In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 47.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.3 MB/s eta 0:00:00


In [2]:

!pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from tabulate import tabulate
from nltk.translate.bleu_score import sentence_bleu
from tqdm.auto import tqdm
import torch
import os

In [4]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import string
import random
from tqdm.auto import tqdm
import torch
import transformers
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from transformers import AdamW, get_cosine_schedule_with_warmup
from transformers import pipeline, set_seed

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import os
folder_path = '/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning'
for filename in os.listdir(folder_path):
    print(filename)

Finetuned_GPT2_10_epochs_V0.pt
FineTune_gpt2_engine_type_5_epochs.zip
FineTune_gpt2_drive_type_5_epochs.zip
FineTune_gpt2_question_answering_5_epochs.zip
FineTune_gpt2_question_answering_engine_type_And_Drive_Type.zip
FineTune_gpt2_drive_type_And_Engine_type_5_epochs.zip
FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS.zip
FineTune_gpt2_Distilled_version_10_epochs.zip
FineTune_gpt2_engine_type_And_Drive_Type_prunned_50.zip
FineTune_gpt2_engine_type_And_Drive_Type_prunned_10.zip
Regression
FineTune_gpt2_engine_type_And_Drive_Type_10_epochs.zip


In [7]:
import torch
# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Define the path to your GPT-2 model file
model_path_1 = '/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/Finetuned_GPT2_10_epochs_V0.pt'

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_Distilled_version_10_epochs.zip" -d FineTune_gpt2_Distilled_version_10_epochs



Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_Distilled_version_10_epochs.zip
replace FineTune_gpt2_Distilled_version_10_epochs/generation_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_And_Drive_Type_10_epochs.zip" -d FineTune_gpt2_engine_type_And_Drive_Type_10_epochs



Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_And_Drive_Type_10_epochs.zip
replace FineTune_gpt2_engine_type_And_Drive_Type_10_epochs/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
FineTune_gpt2_engine_type_And_Drive_Type_10_epochs = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_engine_type_And_Drive_Type_10_epochs')
FineTune_gpt2_engine_type_And_Drive_Type_10_epochs.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
Model_FineTune_gpt2_Distilled_version_10_epochs = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_Distilled_version_10_epochs')
Model_FineTune_gpt2_Distilled_version_10_epochs.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
Model_FineTune_gpt2_Distilled_version_10_epochs = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_Distilled_version_10_epochs')
Model_FineTune_gpt2_Distilled_version_10_epochs.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("distilgpt2")

model_dist = AutoModelForCausalLM.from_pretrained("distilgpt2")
model_dist.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
print(device)

cuda


In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_5_epochs.zip" -d model_FineTune_Engine_type

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_5_epochs.zip
replace model_FineTune_Engine_type/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_5_epochs.zip" -d model_FineTune_Drive_type

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_5_epochs.zip
replace model_FineTune_Drive_type/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_drive_type_And_Engine_type_5_epochs.zip" -d FineTune_gpt2_drive_type_And_Engine_type_5_epochs
Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_drive_type_And_Engine_type_5_epochs')
Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs.to(device)

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_drive_type_And_Engine_type_5_epochs.zip
replace FineTune_gpt2_drive_type_And_Engine_type_5_epochs/config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS.zip" -d FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS
FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS')
FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS.to(device)

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS.zip
replace FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS/config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [8]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_question_answering_engine_type_And_Drive_Type.zip" -d FineTune_gpt2_question_answering_engine_type_And_Drive_Type
FineTune_gpt2_question_answering_engine_type_And_Drive_Type = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_question_answering_engine_type_And_Drive_Type')
FineTune_gpt2_question_answering_engine_type_And_Drive_Type.to(device)

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_question_answering_engine_type_And_Drive_Type.zip
  inflating: FineTune_gpt2_question_answering_engine_type_And_Drive_Type/config.json  
  inflating: FineTune_gpt2_question_answering_engine_type_And_Drive_Type/generation_config.json  
  inflating: FineTune_gpt2_question_answering_engine_type_And_Drive_Type/pytorch_model.bin  


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_question_answering_5_epochs.zip" -d FineTune_gpt2_question_answering_5_epochs
model_FineTune_gpt2_question_answering_5_epochs = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_question_answering_5_epochs')
model_FineTune_gpt2_question_answering_5_epochs.to(device)

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_question_answering_5_epochs.zip
replace FineTune_gpt2_question_answering_5_epochs/config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
model_FineTune_Engine_type = GPT2LMHeadModel.from_pretrained('/content/model_FineTune_Engine_type')
model_FineTune_Engine_type.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
model_FineTune_Drive_type = GPT2LMHeadModel.from_pretrained('/content/model_FineTune_Drive_type')
model_FineTune_Drive_type.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_And_Drive_Type_prunned_50.zip" -d FineTune_gpt2_engine_type_And_Drive_Type_prunned_50
FineTune_gpt2_engine_type_And_Drive_Type_prunned_50 = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_engine_type_And_Drive_Type_prunned_50')
FineTune_gpt2_engine_type_And_Drive_Type_prunned_50.to(device)

Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_And_Drive_Type_prunned_50.zip
replace FineTune_gpt2_engine_type_And_Drive_Type_prunned_50/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Some weights of the model checkpoint at /content/FineTune_gpt2_engine_type_And_Drive_Type_prunned_50 were not used when initializing GPT2LMHeadModel: ['transformer.wte.weight_orig', 'transformer.h.5.mlp.c_fc.weight_mask', 'transformer.h.10.attn.c_attn.weight_orig', 'transformer.h.0.ln_1.weight_mask', 'transformer.h.7.ln_2.weight_mask', 'transformer.h.7.attn.c_proj.weight_orig', 'transformer.h.10.mlp.c_fc.weight_mask', 'transformer.h.6.attn.c_proj.weight_mask', 'transformer.h.6.mlp.c_fc.weight_mask', 'transformer.h.4.ln_1.weight_orig', 'transformer.h.9.attn.c_attn.weight_orig', 'transformer.h.2.attn.c_attn.weight_mask', 'transformer.h.0.attn.c_proj.weight_mask', 'transformer.h.1.mlp.c_fc.weight_orig', 'transformer.h.6.ln_1.weight_mask', 'transformer.h.6.attn.c_proj.weight_orig', 'transformer.h.0.attn.c_attn.weight_mask', 'transformer.h.9.mlp.c_proj.weight_orig', 'transformer.h.9.ln_1.weight_mask', 'transformer.h.8.mlp.c_proj.weight_orig', 'transformer.h.11.attn.c_attn.weight_mask', 'tra

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
! unzip "/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_And_Drive_Type_prunned_10.zip" -d FineTune_gpt2_engine_type_And_Drive_Type_prunned_10
FineTune_gpt2_engine_type_And_Drive_Type_prunned_10 = GPT2LMHeadModel.from_pretrained('/content/FineTune_gpt2_engine_type_And_Drive_Type_prunned_10')
FineTune_gpt2_engine_type_And_Drive_Type_prunned_10.to(device)


Archive:  /content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Models/NLP_Finetuning/FineTune_gpt2_engine_type_And_Drive_Type_prunned_10.zip
replace FineTune_gpt2_engine_type_And_Drive_Type_prunned_10/generation_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Some weights of the model checkpoint at /content/FineTune_gpt2_engine_type_And_Drive_Type_prunned_10 were not used when initializing GPT2LMHeadModel: ['transformer.wte.weight_orig', 'transformer.h.5.mlp.c_fc.weight_mask', 'transformer.h.10.attn.c_attn.weight_orig', 'transformer.h.0.ln_1.weight_mask', 'transformer.h.7.ln_2.weight_mask', 'transformer.h.7.attn.c_proj.weight_orig', 'transformer.h.10.mlp.c_fc.weight_mask', 'transformer.h.6.attn.c_proj.weight_mask', 'transformer.h.6.mlp.c_fc.weight_mask', 'transformer.h.4.ln_1.weight_orig', 'transformer.h.9.attn.c_attn.weight_orig', 'transformer.h.2.attn.c_attn.weight_mask', 'transformer.h.0.attn.c_proj.weight_mask', 'transformer.h.1.mlp.c_fc.weight_orig', 'transformer.h.6.ln_1.weight_mask', 'transformer.h.6.attn.c_proj.weight_orig', 'transformer.h.0.attn.c_attn.weight_mask', 'transformer.h.9.mlp.c_proj.weight_orig', 'transformer.h.9.ln_1.weight_mask', 'transformer.h.8.mlp.c_proj.weight_orig', 'transformer.h.11.attn.c_attn.weight_mask', 'tra

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
import logging
# Set the logging level to ERROR for the transformers library
logging.getLogger("transformers").setLevel(logging.ERROR)

In [ ]:
model_loaded = GPT2LMHeadModel.from_pretrained('gpt2')
model_loaded.to(device)
model_loaded.load_state_dict(torch.load(model_path_1))

<All keys matched successfully>

In [ ]:
from rouge import Rouge

def calculate_scores(model, tokenizer, data):
    bleu_scores = []
    rouge_scores = []
    perplexities = []
    rouge = Rouge()

    for index, row in tqdm(data.iterrows(), total=len(data), desc="Calculating scores"):
        # Get the reference text from the 'prompt_target' column
        reference_text = row['prompt_target']

        # Generate the text using the model
        prompt = f"Car: {row['Full Name']}\ndescription:"
        generated_text = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to(device)
        output_text = model.generate(
            generated_text,
            max_length=250,
            temperature=0.005,
            do_sample=False,
            top_k=100,
            top_p=0.95,
            repetition_penalty=1.2,
            num_return_sequences=1,
        )
        decoded_output = tokenizer.decode(output_text[0], skip_special_tokens=True)

        # Calculate the BLEU score
        bleu_score = sentence_bleu([reference_text], decoded_output)
        bleu_scores.append(bleu_score)

        # Calculate the Rouge score
        rouge_score = rouge.get_scores(decoded_output, reference_text)[0]['rouge-l']['f']
        rouge_scores.append(rouge_score)

        # Calculate the perplexity of the generated text
        input_ids = tokenizer.encode(decoded_output, add_special_tokens=True, return_tensors="pt").to(device)
        with torch.no_grad():
            output = model(input_ids, labels=input_ids)
            loss = output.loss
        perplexity = torch.exp(loss)
        perplexities.append(perplexity.item())

    # Calculate the average scores
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
    avg_rouge_score = sum(rouge_scores) / len(rouge_scores)
    avg_perplexity = sum(perplexities) / len(perplexities)

    # Create a table with the scores
    scores_table = pd.DataFrame({
        'Model': [str(model)],
        'BLEU Score': [avg_bleu_score],
        'Rouge Score': [avg_rouge_score],
        'Perplexity': [avg_perplexity]
    })

    return scores_table

In [ ]:
def evaluate_models(models, tokenizer, data):
    results = []
    for model_name, model in tqdm(models.items(), desc="Evaluating models"):
        scores_table = calculate_scores(model, tokenizer, data)
        avg_bleu_score = scores_table['BLEU Score'][0]
        avg_rouge_score = scores_table['Rouge Score'][0]
        avg_perplexity = scores_table['Perplexity'][0]
        results.append((model_name, avg_bleu_score, avg_rouge_score, avg_perplexity))

    # Sort the results by BLEU score in descending order
    results.sort(key=lambda x: x[1], reverse=True)

    # Print the results in a table format
    headers = ['Model', 'BLEU Score', 'Rouge Score', 'Perplexity']
    table = tabulate(results, headers=headers, tablefmt='orgtbl')
    print(table)

In [ ]:
models = {
    #'Distilled_GPT2': model_dist,
    'FineTune_gpt2_Distilled_version_10_epochs': Model_FineTune_gpt2_Distilled_version_10_epochs,
    'FineTune_gpt2_engine_type_And_Drive_Type_prunned_50': FineTune_gpt2_engine_type_And_Drive_Type_prunned_50,
    'FineTune_gpt2_engine_type_And_Drive_Type_prunned_10': FineTune_gpt2_engine_type_And_Drive_Type_prunned_10,
    #'model GPT2': model,
    'FineTune_gpt2_engine_type_And_Drive_Type_10_epochs' :FineTune_gpt2_engine_type_And_Drive_Type_10_epochs,
    'model_finetune_10_epochs': model_loaded,
    'model_FineTune with_prefinetune_type' : model_FineTune_Engine_type,
    'model_FineTune_with prefinetune_drive_type' : model_FineTune_Drive_type,
    'Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs':Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs,
    'model_FineTune_gpt2_question_answering_5_epochs':model_FineTune_gpt2_question_answering_5_epochs,
    'FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS': FineTune_gpt2_drive_type_And_Engine_type_5_epochs_Instructiontnune_QAS,
    'FineTune_gpt2_question_answering_engine_type_And_Drive_Type': FineTune_gpt2_question_answering_engine_type_And_Drive_Type
}

In [ ]:
models = {
    'FineTune_gpt2_engine_type_And_Drive_Type_10_epochs' :FineTune_gpt2_engine_type_And_Drive_Type_10_epochs,
    'Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs':Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs

}

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/NLU_PROJECT_GTAI_INT/Data/Summary_Final_context.csv")

In [ ]:
data= data.drop(columns=['Pros','Cons','horsepower','engine_type','Torque','Torque_RPM','drive_type','transmission','width','Length','Height'])

In [ ]:
prompts = []
targets = []
li = []
for index, row in data.iterrows():
    carName = str(row['Full Name']).replace('\n', ' ')
    description = str(row['final_context']).replace('\n', ' ')
    prompt = 'Car: ' + carName+'\nDescription: '
    target = description
    targets.append(target)
    prompts.append(prompt)
    li.append(prompt + ' ' + target)
data['prompt_target'] = li

In [ ]:
subset_data = data.sample(frac=0.2, random_state=42)

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [ ]:
evaluate_models(models, tokenizer, subset_data)

Evaluating models:   0%|          | 0/2 [00:00<?, ?it/s]

Calculating scores:   0%|          | 0/628 [00:00<?, ?it/s]

Calculating scores:   0%|          | 0/628 [00:00<?, ?it/s]

| Model                                                   |   BLEU Score |   Rouge Score |   Perplexity |
|---------------------------------------------------------+--------------+---------------+--------------|
| Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs |     0.315264 |      0.243569 |      29.7149 |
| FineTune_gpt2_engine_type_And_Drive_Type_10_epochs      |     0.312435 |      0.242218 |      29.4719 |


In [ ]:
evaluate_models(models, tokenizer, subset_data)

In [ ]:
evaluate_models(models, tokenizer, subset_data)

Evaluating models:   0%|          | 0/11 [00:00<?, ?it/s]

Calculating scores:   0%|          | 0/314 [00:00<?, ?it/s]

Calculating scores:   0%|          | 0/314 [00:00<?, ?it/s]

Calculating scores:   0%|          | 0/314 [00:00<?, ?it/s]

Calculating scores:   0%|          | 0/314 [00:00<?, ?it/s]

...

Calculating scores:   0%|          | 0/314 [00:00<?, ?it/s]

Calculating scores:   0%|          | 0/314 [00:00<?, ?it/s]

Calculating scores:   0%|          | 0/314 [00:00<?, ?it/s]

Calculating scores:   0%|          | 0/314 [00:00<?, ?it/s]

| Model                                                                  |   BLEU Score |   Rouge Score |   Perplexity |
|------------------------------------------------------------------------+--------------+---------------+--------------|
| Model_FineTune_gpt2_drive_type_And_Engine_type_5_epochs                |    0.299585  |     0.244092  |     29.6804  |
| FineTune_gpt2_engine_type_And_Drive_Type_10_epochs                     |    0.298525  |     0.241548  |     29.3276  |
| model_finetune_10_epochs                                               |    0.285336  |     0.248342  |     23.13    |
| model_FineTune with_prefinetune_type                                   |    0.267366  |     0.235851  |     22.7909  |
| model_FineTune_with prefinetune_drive_type                             |    0.266886  |     0.235814  |     23.1435  |
| model_FineTune_gpt2_question_answering_5_epochs                        |    0.265308  |     0.318292  |      6.83741 |
| FineTune_gpt2_question_answeri

In [18]:
model= FineTune_gpt2_question_answering_engine_type_And_Drive_Type
prompt = """Car: Jeep Wrangler 1998
description:"""
generated_text = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to(device)
output_text = model.generate(
            generated_text,
            max_length=500,
            temperature=0.005,
            do_sample=True,
            top_k=100,
            top_p=0.95,
            repetition_penalty=1.2,
            num_return_sequences=1,
        )
decoded_output = tokenizer.decode(output_text[0], skip_special_tokens=True)
print(decoded_output)

Car: Jeep Wrangler 1998
description:  The jeep wrangler 1998 is a vehicle with a Inline 4 engine that delivers 142 horsepower and 155 lb-ft torque at 3,000. It has four wheel drive  and a 5 on/off switch. The car's dimensions are 177 in length; 64.7 in. in height; 66.9 overall width. Additionally, the base MSRP of this car isn't available yet because of an issue concerning availability or availability of certain packages.  Now an in depth review Of The Jeeps Wranglers 1998!!!!!!!!!!!!!"!!!!!)!!'s debut was to be followed by two new models for '98. A Sport model debuted as a 1999 model. An Outback Wagoneer arrived midyear, sporting standard antilock brakes, a six-speed manual transmission (with automatic shift control), leather seating, a CD player, rear disc jacks and a chrome grille. Two more colors were added last year, Silverstone Metallic and Deep Sandstone Metallic which are now available as options on all trim levels. With its powerful V6 engines under the hood it can tow up 6,50